In [10]:
import cv2
import os
import numpy as np

In [11]:
# 폴더에서 X, y 데이터 불러옴

crop_size = 60

images = []
labels = []

folder_paths = ["./able_with_rotation", "./able_with_rotation_toneup"]

for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        full_path = os.path.join(folder_path, filename)
        able_img = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        if able_img is not None and able_img.shape == (80, 80):
            h, w = able_img.shape
            center_x, center_y = w // 2, h // 2
            half_crop_size = crop_size // 2
            
            start_x = center_x - half_crop_size
            start_y = center_y - half_crop_size
            end_x = center_x + half_crop_size
            end_y = center_y + half_crop_size

            cropped_image = able_img[start_y:end_y, start_x:end_x]
            cropped_image = cv2.resize(cropped_image, dsize=(50, 50))

            images.append(cropped_image)
            labels.append(int(filename[-5]))


images = np.array(images)
labels = np.array(labels)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

# CNN 모델 정의
model = models.Sequential([
    # Convolutional Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(50, 50, 1)),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Layer 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # Flattening Layer
    layers.Flatten(),

    # Fully Connected Layer 1
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),

    # Fully Connected Layer 2
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),

    # Output Layer
    layers.Dense(3, activation='softmax')
])

# 모델 요약 출력
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 50, 50, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 25, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 25, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │       589,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 610,883 (2.33 MB)

 Trainable params: 610,883 (2.33 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
from sklearn.model_selection import KFold

k = 5
kf = KFold(n_splits=k, shuffle=True)

fold_no = 1
accuracies = []

for train_index, val_index in kf.split(X_train):
    print(f'Training fold {fold_no}')

    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    history = model.fit(X_train_fold, y_train_fold, epochs=20, validation_data=(X_val_fold, y_val_fold))

    scores = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    print(f'Fold {fold_no} accuracy: {scores[1]}')
    accuracies.append(scores[1])

    fold_no += 1

print(f'Average accuracy across all folds: {np.mean(accuracies)}')

Training fold 1
Epoch 1/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.3924 - loss: 16.1585 - val_accuracy: 0.3832 - val_loss: 1.0761
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.4238 - loss: 1.0708 - val_accuracy: 0.5617 - val_loss: 1.0247
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.4788 - loss: 1.0375 - val_accuracy: 0.6010 - val_loss: 0.9503
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.5261 - loss: 0.9724 - val_accuracy: 0.6220 - val_loss: 0.8634
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.5768 - loss: 0.9259 - val_accuracy: 0.6903 - val_loss: 0.7703
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.5968 - loss: 0.8801 - val_accuracy: 0.7559 - val_loss: 0.7115
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.6514 - loss: 0.7564 - val_accuracy: 0.7585 - val_loss: 0.6526
Epoch 8/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.7163 - loss: 0.6598 - val_acc

In [16]:
def evaluate_model(model, X_test, y_test):
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"Test loss: {test_loss:.4f}")
    print(f"Test accuracy: {test_accuracy:.4f}")

    return test_loss, test_accuracy

In [17]:
test_loss, test_accuracy = evaluate_model(model, X_test, y_test)
print(f'test loss : {test_loss}')
print(f'test accuracy : {test_accuracy}')
print(X_test.shape)

15/15 - 0s - 13ms/step - accuracy: 0.9497 - loss: 0.3628
Test loss: 0.3628
Test accuracy: 0.9497
test loss : 0.36278286576271057
test accuracy : 0.9496855139732361
(477, 50, 50)


In [18]:
rotation_test_image = cv2.imread("./test_image_rotation/test_image.png", cv2.IMREAD_GRAYSCALE)
rotation_test_image = cv2.resize(rotation_test_image, dsize=(50, 50))
rotation_test_image = np.array([rotation_test_image])
model.predict(rotation_test_image)  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


array([[1.6694572e-03, 2.6738351e-07, 9.9833030e-01]], dtype=float32)

In [19]:
model.save('CNN_rotation_model.keras')
print('model saved!')

model saved!
